# Setup

In [ ]:
import pandas as pd
import numpy as np
# wizualizacje
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import itertools

from sklearn.metrics import mutual_info_score, adjusted_mutual_info_score, normalized_mutual_info_score
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

# ustawienie szerokiego ekranu wyświetlacza JNotebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# opcje wyświetlania w Pandas
# - maks. 55 kolumn
# - maks. 101 wierszy
# - liczby w notacji dziesiętnej z czterema zerami po przecinku
pd.set_option('display.max_columns', 55)
pd.set_option('display.max_rows', 101)
pd.set_option('display.float_format', lambda x: f"{x:.4f}")

# opcje formatowania wykresów matplotlib
# - etykiety osi: bold
# - tekst: bold
# - domyślny rozmiar fontu=14
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['font.size'] = '14'

# Wczytanie danych

In [ ]:
filepath = r""
df = (
    pd.read_csv(filepath, sep=';')
    
    .assign(
        **{
             k: lambda df_, col = k: pd.to_datetime(df_[col])
                for k in [
                    'Kiedy jechał*ś?'
                ]
        },
        **{ k: lambda df_, col = k: pd.to_numeric(df_[col], downcast='float')
                for k in [
                    'Koszt podróży:'
                ]
        },
        **{ k: lambda df_, col = k: pd.to_numeric(df_[col], downcast='integer')
                for k in [
                    'Liczba minut opóźnienia na starcie:', 'Liczba minut opóźnienia na mecie:'
                ]
        }
    )
    
    .astype({
        **{ k: 'string' 
               for k in [
                   'Stacja początkowa:', 'Stacja końcowa:', 'Jak wrażenia z podróży? :D (nieobowiązkowe)', 'Uwagi (nieobowiązkowe):'
               ]            
        },
        **{ k: 'category' 
               for k in [
                   'Rodzaj pociągu:', 'Pora dnia:'
               ]            
        }
    })
    
    .rename(columns={
        'Kiedy jechał*ś?'         : 'data_przejazdu',
        'Stacja początkowa:'      : 'stacja_start',
        'Stacja końcowa:'         : 'stacja_finisz',
        'Rodzaj pociągu:'         : 'rodzaj_pociagu',
        'Uwagi (nieobowiązkowe):' : 'uwagi',
        'Koszt podróży:'          : 'cena_biletu',
        'Pora dnia:'              : 'pora_dnia_przejazdu',
        'Liczba minut opóźnienia na starcie:' : 'opoznienie_start',
        'Liczba minut opóźnienia na mecie:'   : 'opoznienie_finisz',
        'Jak wrażenia z podróży? :D (nieobowiązkowe)' : 'wrazenia'
    })
    
    .drop(columns=['wrazenia', 'uwagi'])
    .drop(index=[30, 50])
    
    .assign(
        pora_dnia_przejazdu = lambda df_: np.select(
        [
            (df_.pora_dnia_przejazdu.isnull() & df_.stacja_start.str.contains('Legionowo')).astype(bool),
            (df_.pora_dnia_przejazdu.isnull() & df_.stacja_finisz.str.contains('Legionowo')).astype(bool),
            (df_.pora_dnia_przejazdu.isnull() & df_.stacja_finisz.str.contains('Gdask')).astype(bool)
        ],
        [
            'rano',
            'popołudnie',
            'popołudnie'            
        ],
        df_.pora_dnia_przejazdu
    ))
    
    .reset_index(drop=True)
    
    .assign(
        czy_szkoda = lambda df_: np.select(
            [
                df_.opoznienie_finisz.gt(10) & df_.opoznienie_finisz.le(40),
                df_.opoznienie_finisz.gt(40)
            ],
            [
                1,
                2
            ],
            0
        ),
        
        wyplata_szkoda = lambda df_: np.where(
            df_.opoznienie_finisz.gt(10),
            df_.cena_biletu * np.exp(df_.opoznienie_finisz.div(100)) * df_.czy_szkoda,
            0.0
        )
    )
    
    .sort_values(by='data_przejazdu')
    .reset_index(drop=True)
)
df

1. Stwórz kolumnę przechowującą informację o dniu tygodnia. Stwórz łączną kategorię weekendu (sobota+niedziela+poniedziałek). Stwórz dla wynikowej kolumny własny typ kategorialny, w którym jest zachowana kolejność dni tygodnia.

2. Stwórz kolumnę przechowującą informacje o miesiącu, w którym był przejazd. Sprawdź jej liczebności - czy warto zachować tę zmienną?

3. Stwórz zmienną pora roku. Dołącz do niej typ kategorialny zachowujący kolejność pór roku

4. Zbadaj jakie wartości przyjmuje zmienna stacja_start. Uprość ją według uznania

5. Powtórz operację na zmiennej stacja_finisz

6. Zbadaj w ten sam sposób zmienną rodzaj_pociagu

7. Usuń zbędne kolumny

8. Przepisz zmiany wprowadzone w krokach 1-7 na method chaining rozwijające wczytanie pliku

9. Wyświetl wykres rozrzutu/histogrm zmiennych ciągłych

10. Sprawdź, czy zmienna opóźnienie_start daje sie opisać zależnością wykładniczą?

11. Zwizualizuj związek między wyplatami, a opoznieniem na starcie w zależności od rodzaju pociągu

12. Zwizualizuj związek między wyplatami, a opoznieniem na starcie w zależności od pory dnia

13. Zwizualizuj związek między wyplatami, a opoznieniem na starcie w zależności od stacji końcowej

14. Zwizualizuj związek między wyplatami, a opoznieniem na starcie w zależności od dnia tygodnia przejazdu

15. Zwizualizuj związek między wyplatami, a opoznieniem na starcie w zależności od pory roku

16. Policz informację wzajemną przy użyciu mutual_info_score dla każdej pary zmiennych

17. Zwizualizuj ją przy użyciu heat_map

18. Powtórz powyższe dla adjusted_mutual_info_score

19. Powtór powyższe dla normalized_mutual__infor_score

20. Policz mutula_info przy użyciu metody KNN (mutual_info_classif i mutual_info_regression)

21. Ustaw własne kodowanie koklejności w OrdinalEncoding. Dołącz informację o tym, które zmienne są kategorialne to klasyfikatorów mutual_info

22. Skopiuj kod z poprzedniej komórki i uruchom go ponownie

23. Dorzuć do kodu informację o stanie wyjściowym random_state